In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.exceptions import SSLError, ConnectionError

/Users/fl21879/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# load csv file
legal_df = pd.read_csv("likely_legal.csv")
print(f"Number of rows in legal_df: {len(legal_df)}")
legal_df.head()
legal_df_10 = legal_df.iloc[:10] # testing with only first 10 rows
legal_df_10

Number of rows in legal_df: 1573


,parent_url,postcodes,page_count,legal_term_count,unique_legal_terms,legal_density,z_score,chi_squared,p_value,total_words
0,lapg.co.uk,['SE1 7AA'],6,1441,28,0.0421,68.922,21232.931,0.0,34262
1,www.mplaw.co.uk,"['CM15 8NR', 'RM1 3PJ', 'RM14 2SJ'], ['CM15 8N...",6,1261,31,0.0355,60.282,15299.613,0.0,35552
2,www.ftbchambers.co.uk,"['EC4Y 7BY'], ['SW7 4DN', 'NW3 1PD', 'YO17 9DP...",3,1156,34,0.0316,55.242,12270.317,0.0,36561
3,tannlaw.co.uk,['CV1 2NU'],6,1111,32,0.0383,53.082,14715.200,0.0,29018
4,www.bishopslaw.co.uk,"['SP1 1LY', 'SP10 2AA', 'SP6 1AZ', 'SP4 7AW'],...",6,1010,18,0.0184,48.233,5500.143,0.0,54770
5,ashwoodsolicitors.co.uk,"['BB2 2DH', 'M14 5TB'], ['M14 5TB', 'BB2 2DH']",8,1008,18,0.0490,48.137,17602.474,0.0,20589
6,litkraftsolicitors.co.uk,"['S9 1TP'], ['SK9 5AF', 'S9 1TP']",2,1007,19,0.1330,48.089,51118.102,0.0,7571
7,www.gardencourtchambers.co.uk,"['B30 9EB', 'SK9 5AF', 'WC2A 3LJ'], ['WC2A 3LJ']",5,951,28,0.0668,45.401,23306.300,0.0,14246
8,www.forbessolicitors.co.uk,"['BB1 8DD', 'BB2 9QY'], ['BB2 9QY', 'BB1 8DD']",10,930,21,0.0615,44.393,20872.694,0.0,15111
9,www.gulbenkian.co.uk,['WC2H 8DJ'],6,910,27,0.0226,43.433,6423.906,0.0,40314


In [3]:
# add dataframe columns for selected HTML tags content
selected_tags = ['address', 'title', 'header', 'footer']
for tag in selected_tags:
    legal_df_10.loc[:,tag + "_tag"] = ""
legal_df_10

/var/folders/mj/m740m_294yx3tt2xnqpxvnpm0000gp/T/ipykernel_8901/3711070807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  legal_df_10.loc[:,tag + "_tag"] = ""
/var/folders/mj/m740m_294yx3tt2xnqpxvnpm0000gp/T/ipykernel_8901/3711070807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  legal_df_10.loc[:,tag + "_tag"] = ""
/var/folders/mj/m740m_294yx3tt2xnqpxvnpm0000gp/T/ipykernel_8901/3711070807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,parent_url,postcodes,page_count,legal_term_count,unique_legal_terms,legal_density,z_score,chi_squared,p_value,total_words,address_tag,title_tag,header_tag,footer_tag
0,lapg.co.uk,['SE1 7AA'],6,1441,28,0.0421,68.922,21232.931,0.0,34262,,,,
1,www.mplaw.co.uk,"['CM15 8NR', 'RM1 3PJ', 'RM14 2SJ'], ['CM15 8N...",6,1261,31,0.0355,60.282,15299.613,0.0,35552,,,,
2,www.ftbchambers.co.uk,"['EC4Y 7BY'], ['SW7 4DN', 'NW3 1PD', 'YO17 9DP...",3,1156,34,0.0316,55.242,12270.317,0.0,36561,,,,
3,tannlaw.co.uk,['CV1 2NU'],6,1111,32,0.0383,53.082,14715.200,0.0,29018,,,,
4,www.bishopslaw.co.uk,"['SP1 1LY', 'SP10 2AA', 'SP6 1AZ', 'SP4 7AW'],...",6,1010,18,0.0184,48.233,5500.143,0.0,54770,,,,
5,ashwoodsolicitors.co.uk,"['BB2 2DH', 'M14 5TB'], ['M14 5TB', 'BB2 2DH']",8,1008,18,0.0490,48.137,17602.474,0.0,20589,,,,
6,litkraftsolicitors.co.uk,"['S9 1TP'], ['SK9 5AF', 'S9 1TP']",2,1007,19,0.1330,48.089,51118.102,0.0,7571,,,,
7,www.gardencourtchambers.co.uk,"['B30 9EB', 'SK9 5AF', 'WC2A 3LJ'], ['WC2A 3LJ']",5,951,28,0.0668,45.401,23306.300,0.0,14246,,,,
8,www.forbessolicitors.co.uk,"['BB1 8DD', 'BB2 9QY'], ['BB2 9QY', 'BB1 8DD']",10,930,21,0.0615,44.393,20872.694,0.0,15111,,,,
9,www.gulbenkian.co.uk,['WC2H 8DJ'],6,910,27,0.0226,43.433,6423.906,0.0,40314,,,,


In [4]:
# iterate through URLs in dataframe
for url in legal_df_10["parent_url"]:
    
    # add https:// if url does not start with 'https://' to get valid url
    if url.startswith("https://") == True:
        fetch_url = url
    else:
        fetch_url = "https://" + url

    # get HTML content  
    try:
        response = requests.get(fetch_url)
        soup = BeautifulSoup(response.text, "html.parser")
        
    # skip urls that give SSL or connection errors (websites don't exist)
    except SSLError:
        print(f"SSL Error for URL: {fetch_url}")
        continue
    except ConnectionError:
        print(f"Connection Error for URL: {fetch_url}")
        continue
    
    # get desired HTML tags' content and add to dataframe
    for tag_name in selected_tags:
        found_tags = soup.find_all(tag_name)
        
        tag_text = []
        for t in found_tags:
            # get text from tag, remove extra whitespace
            text = t.get_text(separator = " ", strip = True)
            text = " ".join(text.split())
            if text:
                tag_text.append(text)
        
        # add tag text to dataframe, join multiple tag entries with " | "        
        legal_df_10.loc[legal_df_10["parent_url"] == url, tag_name + "_tag"] = " | ".join(tag_text)

legal_df_10.head()

SSL Error for URL: https://ashwoodsolicitors.co.uk
Connection Error for URL: https://litkraftsolicitors.co.uk


,parent_url,postcodes,page_count,legal_term_count,unique_legal_terms,legal_density,z_score,chi_squared,p_value,total_words,address_tag,title_tag,header_tag,footer_tag
0,lapg.co.uk,['SE1 7AA'],6,1441,28,0.0421,68.922,21232.931,0.0,34262,,403 Forbidden,,
1,www.mplaw.co.uk,"['CM15 8NR', 'RM1 3PJ', 'RM14 2SJ'], ['CM15 8N...",6,1261,31,0.0355,60.282,15299.613,0.0,35552,,Mullis & Peake Solicitors | Essex Lawyers and ...,Call us on 01708 784000 Request a call back Bu...,Accessibility Terms of Use Legal and Privacy C...
2,www.ftbchambers.co.uk,"['EC4Y 7BY'], ['SW7 4DN', 'NW3 1PD', 'YO17 9DP...",3,1156,34,0.0316,55.242,12270.317,0.0,36561,,FTB Chambers,clerks@ftbchambers.co.uk 020 7353 8415,Francis Taylor Building Inner Temple London EC...
3,tannlaw.co.uk,['CV1 2NU'],6,1111,32,0.0383,53.082,14715.200,0.0,29018,,Tann Law Solicitors | Trusted Solicitors In We...,Mon - FRI: 9:00am - 5:00pm info@tannlaw.co.uk ...,Back to top Call Us 024 7763 2323 Mobile 07915...
4,www.bishopslaw.co.uk,"['SP1 1LY', 'SP10 2AA', 'SP6 1AZ', 'SP4 7AW'],...",6,1010,18,0.0184,48.233,5500.143,0.0,54770,,403 Forbidden,,


In [5]:
# save dataframe to csv file
legal_df_10.to_csv("legal_html.csv", index = False)